In [ ]:
#INTEGRANTES: LUISA MARÍA HERNÁNDEZ QUINTERO, CAROLINA TABORDA VARGAS
import pandas as pd
import numpy as np
from scipy import stats
import statistics
import matplotlib.pyplot as plt
# Leer archivo Excel y usar la segunda fila como encabezado
df = pd.read_excel("FLIR_groups1and2.xlsx", header=1)

# Verificar que es un DataFrame
         
print(df.head())


In [ ]:
#Análisis exploratorio de datos

#Promedio de la variable Max1R13_
df = pd.read_excel("FLIR_groups1and2.xlsx", header=2)
columna=df[['Max1R13_1', 'Max1R13_2', 'Max1R13_3', 'Max1R13_4']].mean(axis=1)
df_promedio = pd.DataFrame(columna, columns=['promedio_max'])
print(f"El promedio de Max1R13_ es: \n {df_promedio}")

#Tipos de datos: Variables numéricas y Variables categóricas
variables_numericas = ['aveOralM', 'Age', 'T_atm', 'Humidity', 'Max1R13_']
variables_categoricas = ['Gender', 'Ethnicity', 'Cosmetics']



In [89]:
#Construcción de un DataFrame

#Tipos de datos: Variables numéricas y Variables categóricas

list_1=df['aveOralM'].tolist()
list_2=df['Gender'].tolist()
list_3=df['Age'].tolist()
list_4=df['Ethnicity'].tolist()
list_5=df['T_atm'].tolist()
list_6=df['Humidity'].tolist()
list_7 = df['Cosmetics'].map({1: "Cosmético aplicado", 0: "No aplicado"}).tolist()
list_8=df_promedio['promedio_max'].tolist()

def valores_estadisticos(valores):
    # Convertir a numpy array por seguridad
    valores = np.array(valores)

    # Si no son numéricos (excepto moda), devolver NaN
    if not np.issubdtype(valores.dtype, np.number):
        media = mediana = sd = mad = varianza = iqr = cv = cvm = np.nan
        try:
            moda = statistics.mode(valores)
        except statistics.StatisticsError:
            moda = 'No única'
    else:
        media = np.mean(valores)
        mediana = np.median(valores)
        try:
            moda = statistics.mode(valores)
        except statistics.StatisticsError:
            moda = 'No única'
        sd = np.std(valores, ddof=1)
        mad = np.mean(np.abs(valores - np.mean(valores)))
        varianza = np.var(valores, ddof=1)
        iqr = np.percentile(valores, 75) - np.percentile(valores, 25)
        cv = sd / media if media != 0 else np.nan
        cvm = mad / media if media != 0 else np.nan

    return [media, mediana, moda, sd, mad, varianza, iqr, cv, cvm]
      
variables = {
    'aveOralM': list_1,
    'Gender': list_2,
    'Age': list_3,
    'Ethnicity': list_4,
    'T_atm': list_5,
    'Humidity': list_6,
    'Cosmetics': list_7,
    'promedio_Max1R13_': list_8,

}

# Calculamos los valores estadísticos por variable
estadisticas = {nombre: valores_estadisticos(lista) for nombre, lista in variables.items()}

# Convertimos a DataFrame con los nombres de filas
estadisticas_df = pd.DataFrame(estadisticas, index=["Media", "Mediana", "Moda", "SD", "MAD", "Varianza", "IQR", "CV", "CVM"])

# Mostramos el resultado
print(estadisticas_df)






KeyError: 'Gender'

Calculo del coeficiente de variación de los datos 

In [ ]:
#no sirve 
azon_sd_mad = estadisticas_df.loc['SD'] /estadisticas_df.loc['MAD']
razon_sd_media = estadisticas_df.loc['SD'] /estadisticas_df.loc['Media']*100
print("Razón SD/MAD:\n")
print(razon_sd_mad)
print("Razón SD/Media:\n")
print(razon_sd_media)


In [ ]:

def detectar_outliers(lista):
    datos = np.array(lista)
    
    # Calcular Q1, Q3 e IQR
    Q1 = np.percentile(datos, 25)
    Q3 = np.percentile(datos, 75)
    IQR = Q3 - Q1

    # Límites inferior y superior
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Detectar valores fuera del rango
    outliers = datos[(datos < limite_inferior) | (datos > limite_superior)]

    # Verificar si hay atípicos
    hay_atipicos = len(outliers) > 0
    cantidad = len(outliers)

    return outliers, hay_atipicos, cantidad



outliers_1, hay_atipicos_1, cantidad_1 = detectar_outliers(list_1)
print("Outliers encontrados en aveOralM  :", outliers_1)
print("¿Hay datos atípicos en aveOralM ?", hay_atipicos_1)
print("Cantidad de datos atípicos en aveOralM :", cantidad_1)

outliers_5, hay_atipicos_5, cantidad_5 = detectar_outliers(list_5)
print("Outliers encontrados en list_5:", outliers_5)
print("¿Hay datos atípicos en list_5?", hay_atipicos_5)
print("Cantidad de datos atípicos en list_5:", cantidad_5)

outliers_6, hay_atipicos_6, cantidad_6 = detectar_outliers(list_6)
print("Outliers encontrados en list_6:", outliers_6)
print("¿Hay datos atípicos en list_6?", hay_atipicos_6)
print("Cantidad de datos atípicos en list_6:", cantidad_6)

outliers_8, hay_atipicos_8, cantidad_8 = detectar_outliers(list_8)
print("Outliers encontrados en list_7:", outliers_8)
print("¿Hay datos atípicos en list_7?", hay_atipicos_8)
print("Cantidad de datos atípicos en list_7:", cantidad_8)



Outliers encontrados en aveOralM  : [37.79 36.14 35.99 37.79 37.99 38.69 38.54 38.59 36.09 37.79 38.09 37.99
 36.14 35.64 36.19 40.34 37.99 38.39 38.04 37.99 37.94 37.99 38.44 39.64
 35.94 36.19 36.19 38.84 38.49 37.89 38.29 38.44 37.74 35.99 37.94 36.19
 38.44 38.29 37.74 37.94 37.74 35.84 36.04 38.94 39.09 38.69 38.34 36.19
 37.94 36.19 36.19 36.09 39.84 39.59 38.29 37.69 35.54 38.29 38.84 38.64
 37.94 38.09 37.79 38.14 38.39 37.69 39.34 37.89 37.89 37.79 36.09 38.74
 37.74 38.14 38.49 38.69 38.54 37.79 39.14 37.99 37.74 38.24 37.94 37.74
 39.54 38.84 38.19 38.04 37.94 38.49 37.69 38.19 37.94 39.09 37.74 38.44
 39.24 38.14 38.94 37.89 37.94 37.89 37.94 38.39 37.94 37.69 38.74 37.79]
¿Hay datos atípicos en aveOralM ? True
Cantidad de datos atípicos en aveOralM : 108
Outliers encontrados en list_5: [27.  26.7 26.8 26.9 27.2 27.2 26.9 21.4 21.2 21.4 21.4 20.5 21.2 21.4
 21.2 21.3 20.8 20.6 21.  21.1 20.8 20.8 20.7 21.  21.1 20.6 20.2 20.3
 20.6 21.  21.  21.  21.3 26.8 26.8 27.  27.  27

In [ ]:

import matplotlib.pyplot as plt

def graficar_histograma(datos, bins=10, titulo="Histograma", xlabel="Valor", ylabel="Frecuencia"):
    """
    Grafica un histograma a partir de una lista de datos.

    Parámetros:
    - datos: lista de valores numéricos
    - bins: número de divisiones del histograma (por defecto 10)
    - titulo: título del gráfico
    - xlabel: etiqueta del eje X
    - ylabel: etiqueta del eje Y
    """
    plt.figure(figsize=(8, 5))
    plt.hist(datos, bins=bins, edgecolor='black', alpha=0.7)
    plt.title(titulo)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()
    
graficar_histograma(list_1, bins=10, titulo="Histograma de aveOralM")
graficar_histograma(list_5, bins=10, titulo="Histograma de T_atm")
graficar_histograma(list_6, bins=10, titulo="Histograma de Humidity")
graficar_histograma(list_8, bins=10, titulo="Histograma de promedio de Max1R13_")


In [ ]:

# Supongamos que tus listas numéricas son estas:
variables = [
    list_1,  # aveOralM
    list_5,  # T_atm
    list_6,  # Humidity
    list_8   # promedio Max1R13
]

nombres = ['aveOralM', 'T_atm', 'Humidity', 'promedio Max1R13']

# Crear gráfico de cajas
plt.figure(figsize=(10, 6))
plt.boxplot(variables, labels=nombres, patch_artist=True)

plt.title("Box-cox de Variables Numéricas")
plt.ylabel("Valores")
plt.grid(True, linestyle="--", alpha=0.5)
plt.show()

In [ ]:
#Gráficos de barras
import matplotlib.pyplot as plt
from collections import Counter

def graficar_categorias_desde_lista(lista, titulo="Frecuencia por categoría", color='lightgreen'):
    """
    Genera una gráfica de barras para una lista de datos categóricos.
    """
    conteo = Counter(lista)
    
    categorias = list(conteo.keys())
    frecuencias = list(conteo.values())

    x = list(range(len(categorias)))  # convertir las categorías a índices numéricos

    plt.figure(figsize=(10, 6))
    plt.bar(x, frecuencias, color=color, edgecolor='black')
    plt.xticks(ticks=x, labels=categorias, rotation=45)
    plt.title(titulo)
    plt.xlabel("Categoría")
    plt.ylabel("Frecuencia")

    for i, valor in enumerate(frecuencias):
        plt.text(i, valor + 0.5, str(valor), ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    plt.show()

graficar_categorias_desde_lista(list_2, 'Gender',color='lightgreen' )
graficar_categorias_desde_lista(list_3, 'Age',color='lightblue' )
graficar_categorias_desde_lista(list_4, 'Ethnicity',color='lightpink' )
graficar_categorias_desde_lista(list_7, 'Cosmetics',color='lavender')

In [ ]:
#Matriz de correlación
import seaborn as sns

variables = {
    'aveOralM': list_1,
    'T_atm': list_5,
    'Humidity': list_6,
    'promedio_Max1R13_': list_8,
}

# Convertir a DataFrame
df = pd.DataFrame(variables)

# Calcular matriz de correlación
matriz_corr = df.corr()

# Mostrar la matriz
print("Matriz de correlación:")
print(matriz_corr)

# Visualizar con mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(matriz_corr, annot=True, cmap='Spectral', fmt=".2f")
plt.title("Matriz de Correlación entre Variables Numéricas")
plt.show()

Punto 2 

In [ ]:
#Codigo para obtener data frame de la poblacion
df_1 = pd.read_excel("FLIR_groups1and2.xlsx", header=2)
def promedio_data(df,  p, q, r, s):
    columna = df[[p, q, r, s]].mean(axis=1)
    return columna.tolist()

T_CRmax=promedio_data(df_1, "Max1R13_1", "Max1R13_2", "Max1R13_3", "Max1R13_4")
T_CLmax=promedio_data(df_1, "Max1L13_1", "Max1L13_2","Max1L13_3","Max1L13_4" )
T̅_CRR=promedio_data(df_1, "aveAllR13_1", "aveAllR13_2", "aveAllR13_3","aveAllR13_4" )
T̅_CRL=promedio_data(df_1, "aveAllL13_1", "aveAllL13_2", "aveAllL13_3","aveAllL13_4" )
T_CEmax=promedio_data(df_1, "canthiMax1", "canthiMax2", "canthiMax3","canthiMax4" )
T_FEmax=promedio_data(df_1,"T_FH_Max1","T_FH_Max2","T_FH_Max3","T_FH_Max4")
T_FCmax=promedio_data(df_1,"T_FHC_Max1","T_FHC_Max2","T_FHC_Max3","T_FHC_Max4")
T_max=promedio_data(df_1, "T_Max1","T_Max2","T_Max3","T_Max4")
T_Mmax=promedio_data(df_1,"T_OR_Max1","T_OR_Max2","T_OR_Max3","T_OR_Max4")
T_FC= promedio_data(df_1, "T_FHCC1", "T_FHCC2", "T_FHCC3","T_FHCC4" ) 
T_FR=promedio_data(df_1,"T_FHRC1", "T_FHRC2","T_FHRC3","T_FHRC4" ) 
T_FL=promedio_data(df_1, "T_FHLC1","T_FHLC2","T_FHLC3","T_FHLC4"  )
T_FB=promedio_data(df_1, "T_FHBC1","T_FHBC2","T_FHBC3","T_FHBC4")
T_FT=promedio_data(df_1, "T_FHTC1","T_FHTC2","T_FHTC3","T_FHTC4")

df_resultados = pd.DataFrame({  #data frame con el promedio de los valores del termografo 
    "T_CRmax": T_CRmax,
    "T_CLmax": T_CLmax,
    "T̅_CRR": T̅_CRR,
    "T̅_CRL": T̅_CRL,
    "T_CEmax": T_CEmax,
    "T_FEmax": T_FEmax,
    "T_FCmax": T_FCmax,
    "T_max": T_max,
    "T_Mmax": T_Mmax,
    "T_FC": T_FC,
    "T_FR": T_FR,
    "T_FL": T_FL,
    "T_FB": T_FB,
    "T_FT": T_FT
})
from tabulate import tabulate
print(tabulate(df_resultados, headers='keys', tablefmt='github', showindex=True))




In [ ]:
#Codigo para encontrar estadisticas metricas de la poblacion 
df_resultados_metricas  = {  #Diccionario para metricas 
    "T_CRmax": T_CRmax,
    "T_CLmax": T_CLmax,
    "T̅_CRR": T̅_CRR,
    "T̅_CRL": T̅_CRL,
    "T_CEmax": T_CEmax,
    "T_FEmax": T_FEmax,
    "T_FCmax": T_FCmax,
    "T_max": T_max,
    "T_Mmax": T_Mmax,
    "T_FC": T_FC,
    "T_FR": T_FR,
    "T_FL": T_FL,
    "T_FB": T_FB,
    "T_FT": T_FT
}
# Calcular estadísticas para cada variable
estadisticas_poblacion = {
    nombre: valores_estadisticos(lista) for nombre, lista in df_resultados_metricas.items()
}

# Crear DataFrame con los resultados
estadisticas_poblacion_df = pd.DataFrame(
    estadisticas_poblacion,
    index=["Media", "Mediana", "Moda", "SD", "MAD", "Varianza", "IQR", "CV", "CVM"]
)

# Imprimir resultados
print("Estadísticas descriptivas de la poblacion:\n")
print(tabulate(estadisticas_poblacion_df, headers='keys', tablefmt='github', showindex=True))

In [ ]:
# Muestreo de datos punto 1
# 1. Realizar el muestreo aleatorio simple con n=500
muestra = df_resultados.sample(n=500, random_state=42) #dataframe de la muestra aleatoria (sample selecciona una muestra aleatoria), randome_state = para siempre obtener la muestra arbitraria 
print("Muestreo aleatorio simple con n=500: \n")
print(tabulate(muestra, headers='keys', tablefmt='github', showindex=True))

In [ ]:
#Codigo para estaditicas de la muestra 
variables_muestra = {
    'T_CRmax': muestra['T_CRmax'].tolist(),
    'T_CLmax': muestra['T_CLmax'].tolist(),
    'T̅_CRR': muestra['T̅_CRR'].tolist(),
    'T̅_CRL': muestra['T̅_CRL'].tolist(),
    'T_CEmax': muestra['T_CEmax'].tolist(),
    'T_FEmax': muestra['T_FEmax'].tolist(),
    'T_FCmax': muestra['T_FCmax'].tolist(),
    'T_max': muestra['T_max'].tolist(),
    'T_Mmax': muestra['T_Mmax'].tolist(),
    'T_FC': muestra['T_FC'].tolist(),
    'T_FR': muestra['T_FR'].tolist(),
    'T_FL': muestra['T_FL'].tolist(),
    'T_FB': muestra['T_FB'].tolist(),
    'T_FT': muestra['T_FT'].tolist()
}

# Calcular estadísticas para cada variable
estadisticas_muestra = {
    nombre: valores_estadisticos(lista) for nombre, lista in variables_muestra.items()
}

# Crear DataFrame con los resultados
estadisticas_muestra_df = pd.DataFrame(
    estadisticas_muestra,
    index=["Media", "Mediana", "Moda", "SD", "MAD", "Varianza", "IQR", "CV", "CVM"]
)

# Imprimir resultados
print("Estadísticas descriptivas de la muestra (n=500):\n")
print(tabulate(estadisticas_muestra_df, headers='keys', tablefmt='github', showindex=True))


segundo punto muestreo 